In [1]:
import os
os.chdir("C:/Users/Xtage/OneDrive - Xtage Technologies Private Limited/UTec - Map Detection")
import config
import cv2
import numpy as np

Folder 'C:/Users/Xtage/OneDrive - Xtage Technologies Private Limited/UTec - Map Detection/Output/001_Sample_Maps' already exists.
Folder 'C:/Users/Xtage/OneDrive - Xtage Technologies Private Limited/UTec - Map Detection/Output/001_Sample_Maps/01_Images/001_Edges' already exists.
Folder 'C:/Users/Xtage/OneDrive - Xtage Technologies Private Limited/UTec - Map Detection/Output/001_Sample_Maps/01_Images/002_Hough_Lines' already exists.


In [2]:
gray = cv2.imread(config.dir_prj_ip + "/" + config.prj_id + ".png")
edges = cv2.Canny(gray,50,150,apertureSize = 3)
cv2.imwrite(config.dir_prj_op_edges + "/" + "Edges.png",edges)
minLineLength=100
lines = cv2.HoughLinesP(image=edges,rho=1,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=minLineLength,maxLineGap=80)

a,b,c = lines.shape
for i in range(a):
    cv2.line(gray, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
    cv2.imwrite(config.dir_prj_op_hough + "/" + "HoughLines.png",gray)